LIBRERÍAS

In [5]:
import warnings
warnings.filterwarnings('ignore')

import cv2
import copy
import ExtraccionDatos as ext
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import ShuffleSplit
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import naive_bayes
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.tree import DecisionTreeClassifier

LECTURA DE DATOS

Para la lectura de datos vamos a hacer uso del método crea_dataset que se encuentra en extraccionDatos.py. El tamaño de las imagenes serán igual para todas 260x260.

In [6]:
X_train = []
y_train = []
X_test = []
y_test = []
nombres_animales = ['cat','elephant','horse','spider']

num_train = input("Introduce el numero de imagenes de entrenamiento (100 o 500): ")

# Iterar sobre cada animal
for animal_a_cargar in nombres_animales:
    X_train_nuevo, y_train_nuevo, X_test_nuevo, y_test_nuevo = ext.crea_dataset(animal_a_cargar, num_train)
    # Concatenar resultados
    X_train = np.concatenate((X_train, X_train_nuevo)) if len(X_train) else X_train_nuevo
    y_train = np.concatenate((y_train, y_train_nuevo)) if len(y_train) else y_train_nuevo
    X_test = np.concatenate((X_test, X_test_nuevo)) if len(X_test) else X_test_nuevo
    y_test = np.concatenate((y_test, y_test_nuevo)) if len(y_test) else y_test_nuevo

print('----')
print(X_train.shape)
print(X_test.shape)
print('----')

----
(2000, 260, 260, 3)
(1200, 260, 260, 3)
----


####  NORMALIZACIÓN DE DATOS

In [ ]:
original_shape_train = X_train.shape
original_shape_test = X_test.shape

X_train = X_train.reshape(-1, original_shape_train[-1])
X_test = X_test.reshape(-1, original_shape_test[-1])

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(original_shape_train)
X_test = X_test.reshape(original_shape_test)

### OBTENER VECTORES A COLOR Y EN ESCALA DE GRISES

In [4]:
X_train_RGB = copy.deepcopy(X_train)
X_test_RGB = copy.deepcopy(X_test)

X_train_GRAY = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test_GRAY = np.zeros((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

for i in range(X_train.shape[0]):
    X_train_GRAY[i] = cv2.cvtColor(X_train_RGB[i].astype('uint8'), cv2.COLOR_BGR2GRAY)

for i in range(X_test.shape[0]):
    X_test_GRAY[i] = cv2.cvtColor(X_test_RGB[i].astype('uint8'), cv2.COLOR_BGR2GRAY)

### EXTRACCION DE CARACTERÍSTICAS

Como la extracción del vector de características lleva mucho tiempo hemos pensado en guardarlos en txt y así vec_train y vec_test pueden leer los datos de los txt, cosa que lleva poco tiempo.

Así mismo, para poder comprobar el funcionamiento de cualquiera de las extracciones, hay que saltar al siguiente apartado donde leeremos directamente los vectores para ahorrar el tiempo de ejecución de las extracciones. [Carga de vectores](#carga-vectores)

#### EXTRACCIÓN VECTOR CARACTERÍSTICAS 1

In [ ]:
vector_train_uniforme = []
vector_test_uniforme = []

for i in range(X_train.shape[0]):
    vector_train_uniforme.append(ext.vector_caracteristicas_uniforme(X_train_GRAY[i], 16))
vector_train_uniforme = np.asarray(vector_train_uniforme)

for i in range(X_test.shape[0]):
    vector_test_uniforme.append(ext.vector_caracteristicas_uniforme(X_test_GRAY[i], 16))
vector_test_uniforme = np.asarray(vector_test_uniforme)

# Trato los valores nan por si acaso
vector_train_uniforme[np.isnan(vector_train_uniforme)] = 0
vector_test_uniforme[np.isnan(vector_test_uniforme)] = 0

np.savetxt("vectores/vec_train_uniforme_500", vector_train_uniforme, fmt='%i')
np.savetxt("vectores/vec_test_uniforme_500", vector_test_uniforme, fmt='%i')

In [ ]:
vector_train_color = []
vector_test_color = []

for i in range(len(X_train)):
    vector_train_color.append(ext.vector_caracteristicasRGB(X_train_RGB[i], 16))
vector_train_color = np.asarray(vector_train_color)

for i in range(len(X_test)):
    vector_test_color.append(ext.vector_caracteristicasRGB(X_test_RGB[i], 16))
vector_test_color = np.asarray(vector_test_color)

# Trato los valores nan por si acaso
vector_train_color[np.isnan(vector_train_color)] = 0
vector_test_color[np.isnan(vector_test_color)] = 0

np.savetxt("vectores/vec_train_color_500.txt", vector_train_color, fmt='%i')
np.savetxt("vectores/vec_test_color_500.txt", vector_test_color, fmt='%i')

In [ ]:
vector_train_basico = []
vector_test_basico = []

for i in range(len(X_train)):
    vector_train_basico.append(ext.vector_caracteristicas(X_train_GRAY[i] ,16))
vector_train_basico = np.asarray(vector_train_basico)

for i in range(len(X_test)):
    vector_test_basico.append(ext.vector_caracteristicas(X_test_GRAY[i], 16))
vector_test_basico = np.asarray(vector_test_basico)

# Trato los valores nan por si acaso
vector_train_basico[np.isnan(vector_train_basico)] = 0
vector_test_basico[np.isnan(vector_test_basico)] = 0

np.savetxt("vectores/vec_train_basico_500.txt", vector_train_basico, fmt='%i')
np.savetxt("vectores/vec_test_basico_500.txt", vector_test_basico, fmt='%i')

#### EXTRACCIÓN VECTOR CARACTERÍSTICAS 2

In [ ]:
vector_train_2 = []
vector_test_2 = []

for i in range(len(X_train)):
    vector_train_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_train_GRAY[i], 4)))
vector_train_2 = np.asarray(vector_train_2)

for i in range(len(X_test)):
    vector_test_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_test_GRAY[i], 4)))
vector_test_2 = np.asarray(vector_test_2)

# Trato los valores nan por si acaso
vector_train_2[np.isnan(vector_train_2)] = 0
vector_test_2[np.isnan(vector_test_2)] = 0

np.savetxt("vectores/vec_train_2_N4_500.txt", vector_train_2, fmt='%i')
np.savetxt("vectores/vec_test_2_N4_500.txt", vector_test_2, fmt='%i')

In [ ]:
vector_train_2 = []
vector_test_2 = []

for i in range(len(X_train)):
    vector_train_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_train_GRAY[i], 8)))
vector_train_2 = np.asarray(vector_train_2)

for i in range(len(X_test)):
    vector_test_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_test_GRAY[i], 8)))
vector_test_2 = np.asarray(vector_test_2)

# Trato los valores nan por si acaso
vector_train_2[np.isnan(vector_train_2)] = 0
vector_test_2[np.isnan(vector_test_2)] = 0

np.savetxt("vectores/vec_train_2_N8_500.txt", vector_train_2, fmt='%i')
np.savetxt("vectores/vec_test_2_N8_500.txt", vector_test_2, fmt='%i')

In [ ]:
vector_train_2 = []
vector_test_2 = []

for i in range(len(X_train)):
    vector_train_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_train_GRAY[i], 16)))
vector_train_2 = np.asarray(vector_train_2)

for i in range(len(X_test)):
    vector_test_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_test_GRAY[i], 16)))
vector_test_2 = np.asarray(vector_test_2)

# Trato los valores nan por si acaso
vector_train_2[np.isnan(vector_train_2)] = 0
vector_test_2[np.isnan(vector_test_2)] = 0

np.savetxt("vectores/vec_train_2_N16_500.txt", vector_train_2, fmt='%i')
np.savetxt("vectores/vec_test_2_N16_500.txt", vector_test_2, fmt='%i')

In [ ]:
vector_train_2 = []
vector_test_2 = []

for i in range(len(X_train)):
    vector_train_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_train_GRAY[i], 32)))
vector_train_2 = np.asarray(vector_train_2)

for i in range(len(X_test)):
    vector_test_2.append(np.uint8(255 * ext.vector_caracteristicas2(X_test_GRAY[i], 32)))
vector_test_2 = np.asarray(vector_test_2)

# Trato los valores nan por si acaso
vector_train_2[np.isnan(vector_train_2)] = 0
vector_test_2[np.isnan(vector_test_2)] = 0

np.savetxt("vectores/vec_train_2_N32_500.txt", vector_train_2, fmt='%i')
np.savetxt("vectores/vec_test_2_N32_500.txt", vector_test_2, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 8, 2)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 8, 2)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N8_M2_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N8_M2_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 8, 3)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 8, 3)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N8_M3_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N8_M3_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 8, 4)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 8, 4)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N8_M4_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N8_M4_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 16, 2)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 16, 2)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N16_M2_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N16_M2_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 16, 3)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 16, 3)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N16_M3_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N16_M3_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 16, 4)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 16, 4)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N16_M4_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N16_M4_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 32, 2)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 32, 2)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N32_M2_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N32_M2_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 32, 3)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 32, 3)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N32_M3_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N32_M3_500.txt", vector_test_mxm, fmt='%i')

In [ ]:
vector_train_mxm = []
vector_test_mxm = []

for i in range(len(X_train)):
    vector_train_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_train_GRAY[i], 32, 4)))
vector_train_mxm = np.asarray(vector_train_mxm)

for i in range(len(X_test)):
    vector_test_mxm.append(np.uint8(255 * ext.vector_caracteristicas2_con_MxM(X_test_GRAY[i], 32, 4)))
vector_test_mxm = np.asarray(vector_test_mxm)

# Trato los valores nan por si acaso
vector_train_mxm[np.isnan(vector_train_mxm)] = 0
vector_test_mxm[np.isnan(vector_test_mxm)] = 0

np.savetxt("vectores/vec_train_mxm_N32_M4_500.txt", vector_train_mxm, fmt='%i')
np.savetxt("vectores/vec_test_mxm_N32_M4_500.txt", vector_test_mxm, fmt='%i')

<a id='carga-vectores'></a>

### CARGA VECTORES

De todas estas celdas, ejecutar unicamente aquella que se quiere probar. Una vez ejecutada saltar directamente a los apartados de [Entrenamiento](#indice).
Para ello, evidentemente, se debe de extraer del zip entergado los ficheros correspondientes.

EXTRACCION 1

In [ ]:
'''
Versión básica del primer modelo de extracción junto a su PCA con train 100
'''
vector_train = np.loadtxt('vectores/vec_train_basico_100.txt') 
vector_test = np.loadtxt('vectores/vec_test_basico_100.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión uniforme del primer modelo de extracción junto a su PCA con train 100
'''
vector_train = np.loadtxt('vectores/vec_train_uniform_100.txt') 
vector_test = np.loadtxt('vectores/vec_test_uniform_100.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión color del primer modelo de extracción junto a su PCA con train 100
'''
vector_train = np.loadtxt('vectores/vec_train_color_100.txt') 
vector_test = np.loadtxt('vectores/vec_test_color_100.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión básica del primer modelo de extracción junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_basico_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_basico_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión uniforme del primer modelo de extracción junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_uniform_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_uniform_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión color del primer modelo de extracción junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_color_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_color_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

EXTRACCION 2

In [ ]:
'''
Versión del segundo modelo de extracción con N = 16 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_2_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_2_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con N = 4 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_2_N4_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_2_N4_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con N = 8 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_2_N8_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_2_N8_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con N = 32 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_2_N32_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_2_N32_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 16 y M = 2 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 8 y M = 2 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N8_M2_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N8_M2_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 8 y M = 3 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N8_M3_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N8_M3_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 8 y M = 4 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N8_M4_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N8_M4_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 16 y M = 3 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N16_M3_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N16_M3_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 16 y M = 4 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N16_M4_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N16_M4_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 32 y M = 2 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N32_M2_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N32_M2_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 32 y M = 3 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N32_M3_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N32_M3_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

In [ ]:
'''
Versión del segundo modelo de extracción con modificaciones mxm con N = 32 y M = 4 junto a su PCA con train 500
'''
vector_train = np.loadtxt('vectores/vec_train_mxm_N32_M4_500.txt') 
vector_test = np.loadtxt('vectores/vec_test_mxm_N32_M4_500.txt')

pca_vector_train, pca_vector_test = ext.pca(vector_train, vector_test, 100)

<a id='indice'></a>

#### ENTRENAMIENTO DE MODELOS
<br>&emsp;&emsp;[Regresión logística](#regresion-logistica)
<br>&emsp;&emsp;[Naive Bayes](#naive-bayes)
<br>&emsp;&emsp;[Redes Neuronales](#redes-neuronales)
<br>&emsp;&emsp;[OVA](#OVA)
<br>&emsp;&emsp;[OVO](#OVO)
<br>&emsp;&emsp;[Decision Tree](#decision-tree)
<br>&emsp;&emsp;[Bagging](#bagging)
<br>&emsp;&emsp;[Ada Boost](#ada-boost)
<br>&emsp;&emsp;[Random Forest](#random-forest)

<a id='regresion-logistica'></a>

REGRESIÓN LOGÍSTICA

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
# Genera el diccionario param_grid con los valores indicados
parameter_space = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'solver' : ['newton-cg', 'lbfgs', 'sag']
}
ss = ShuffleSplit(n_splits=1,test_size=0.2,random_state=0)
modelo = LogisticRegression(penalty='l2',max_iter=400)
grid_search = GridSearchCV(modelo, parameter_space, cv=ss)
#Ajustar el modelo utilizando GridSearchCV en los datos de entrenamiento
grid_search.fit(pca_vector_train, y_train.ravel())

#Obtener los mejores hiperparámetros encontrados
best_params_c = grid_search.best_params_['C']
best_params_solver = grid_search.best_params_['solver']

#Obtener el mejor modelo entrenado
Regresion_Logistica = grid_search.best_estimator_

#Calcular la precisión en el conjunto de train y test.
accTrain = metrics.accuracy_score(y_train, Regresion_Logistica.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, Regresion_Logistica.predict(pca_vector_test))

#Imprimimos los resultados obtenidos
print(f'Mejor parámetro c: {best_params_c}')
print(f'Mejor parámetro solver: {best_params_solver}')
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

<a id='naive-bayes'></a>

NAIVE BAYES

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
# Definir el modelo de Naive Bayes
Naive_Bayes = naive_bayes.GaussianNB()
# Entenar el modelo con los datos de train
Naive_Bayes.fit(pca_vector_train, y_train.ravel())
#Calcular la precisión en el conjunto de train y test.
accTrain = metrics.accuracy_score(y_train, Naive_Bayes.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, Naive_Bayes.predict(pca_vector_test))
# Imprimimos los resultados obtenidos
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

<a id='redes-neuronales'></a>

REDES NEURONALES

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
clasificador = MLPClassifier(max_iter=2500, random_state=12)
param_grid_redesNeuronales = {
    'hidden_layer_sizes' : np.arange(1,30,2),
    'alpha': [0.05, 0.1, 0.15],
}
gridRedesNeuronales = GridSearchCV(clasificador, param_grid_redesNeuronales, cv=ShuffleSplit(n_splits=1,test_size=0.2,random_state=0))
gridRedesNeuronales.fit(pca_vector_train, y_train.ravel())
Redes_Neuronales = gridRedesNeuronales.best_estimator_
best_params = gridRedesNeuronales.best_params_ 
accTrain = Redes_Neuronales.score(pca_vector_train, y_train)
accTest = Redes_Neuronales.score(pca_vector_test, y_test)
print(f'Mejores hiperparámetros : {best_params}')
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

<a id='OVA'></a>

OVA

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
clasificador = DecisionTreeClassifier(criterion='entropy', random_state=12)
OVA = OneVsRestClassifier(clasificador)
param_grid_ovo = {'estimator__max_depth': np.arange(1, 10)}
gridOVA = GridSearchCV(OVA, param_grid_ovo, cv=ShuffleSplit(n_splits=1,test_size=0.2,random_state=0))
gridOVA.fit(pca_vector_train, y_train.ravel())
OVA_best = gridOVA.best_estimator_
best_params_ova = gridOVA.best_params_['estimator__max_depth']
accTrain = metrics.accuracy_score(y_train, OVA_best.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, OVA_best.predict(pca_vector_test))
print(f'Mejores hiperparámetros: {best_params_ova}')
print(f'Precisión en train: {accTrain * 100:.3f}%')
print(f'Precisión en test: {accTest * 100:.3f}%')

<a id='OVO'></a>

OVO

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
clasificador = DecisionTreeClassifier(criterion='entropy', random_state=12)
OVO = OneVsOneClassifier(clasificador)
param_grid_ovo = {'estimator__max_depth': np.arange(1, 10)}
gridOVO = GridSearchCV(OVO, param_grid_ovo, cv=ShuffleSplit(n_splits=1,test_size=0.2,random_state=0))
gridOVO.fit(pca_vector_train, y_train.ravel())
OVO_best = gridOVO.best_estimator_
best_params_ovo = gridOVO.best_params_['estimator__max_depth']
accTrain = metrics.accuracy_score(y_train, OVO_best.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, OVO_best.predict(pca_vector_test))
print(f'Mejores hiperparámetros: {best_params_ovo}')
print(f'Precisión en train: {accTrain * 100:.3f}%')
print(f'Precisión en test: {accTest * 100:.3f}%')

<a id='decision-tree'></a>

DECISION TREE

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
# Definir un modelo de clasificación con DecisionTreeClassifier
Decision_Tree = DecisionTreeClassifier(criterion = 'entropy', random_state=12)
# Ajustar el modelo utilizando los datos de entrenamiento
Decision_Tree.fit(pca_vector_train, y_train)
# Calcular la precisión en el conjunto de train y test
accTrain = metrics.accuracy_score(y_train, Decision_Tree.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, Decision_Tree.predict(pca_vector_test))
# Imprimir los resultados obtenidos
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

<a id='bagging'></a>

BAGGING

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
# Definir un modelo de clasificación con BaggingClassifier
modelo_bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion = 'entropy', random_state=12), random_state=12)
param_grid={'base_estimator': [Regresion_Logistica, Naive_Bayes, Redes_Neuronales,Decision_Tree]}
grid_search_bagging = GridSearchCV(modelo_bagging, param_grid, cv= ShuffleSplit(n_splits=1,test_size=0.2,random_state=0))

# Ajustar el modelo utilizando los datos de entrenamiento
grid_search_bagging.fit(pca_vector_train, y_train.ravel())

# Obtener los mejores hiperparámetros y mejor modelo
best_params = grid_search_bagging.best_params_['base_estimator']
Bagging = grid_search_bagging.best_estimator_

# Calcular la precisión en el conjunto de train y test
accTrain = metrics.accuracy_score(y_train, Bagging.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, Bagging.predict(pca_vector_test))
# Imprimir los resultados obtenidos
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

<a id='ada-boost'></a>

ADA-BOOST

<div style="text-align: right;">


###### [Índice modelos](#indice)

</div>

In [ ]:
# Definir un modelo de clasificación con AdaBoostClassifier
base_estimator = DecisionTreeClassifier(criterion='entropy', random_state=12)
Ada_Boost = AdaBoostClassifier(base_estimator=base_estimator, random_state=12)

# Definir los hiperparámetros que deseamos probar
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.001, 0.01, 0.1, 1]
}

# Crear un objeto GridSearchCV
grid_search_ada_boost = GridSearchCV(Ada_Boost, param_grid, cv=ShuffleSplit(n_splits=1,test_size=0.2,random_state=0))

# Ajustar el modelo utilizando GridSearchCV en los datos de entrenamiento
grid_search_ada_boost.fit(pca_vector_train, y_train.ravel())

# Obtener los mejores hiperparámetros y mejor modelo
best_params = grid_search_ada_boost.best_params_
Ada_Boost_best = grid_search_ada_boost.best_estimator_

# Calcular la precisión en el conjunto de train y test
accTrain = metrics.accuracy_score(y_train, Ada_Boost_best.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, Ada_Boost_best.predict(pca_vector_test))

# Imprimir los resultados obtenidos
print(f'Mejores hiperparámetros: {best_params}')
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

<a id='random-forest'></a>

RANDOM FOREST

<div style="text-align: right;">

###### [Índice modelos](#indice)

</div>

In [ ]:
# Definir un modelo de clasificación con RandomForestClassifier
Random_Forest = RandomForestClassifier(criterion='entropy', max_features='log2', random_state=12)

# Definir los hiperparámetros que deseamos probar
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 4]
}

# Crear un objeto GridSearchCV
grid_search_random_forest = GridSearchCV(Random_Forest, param_grid, cv= ShuffleSplit(n_splits=1,test_size=0.2,random_state=0))

# Ajustar el modelo utilizando GridSearchCV en los datos de entrenamiento
grid_search_random_forest.fit(pca_vector_train, y_train.ravel())

# Obtener los mejores hiperparámetros y mejor modelo
best_params = grid_search_random_forest.best_params_
Random_Forest_best = grid_search_random_forest.best_estimator_

# Calcular la precisión en el conjunto de train y test
accTrain = metrics.accuracy_score(y_train, Random_Forest_best.predict(pca_vector_train))
accTest = metrics.accuracy_score(y_test, Random_Forest_best.predict(pca_vector_test))

# Imprimir los resultados obtenidos
print(f'Mejores hiperparámetros: {best_params}')
print(f'Precision en train: {accTrain * 100:.3f}%')
print(f'Precision en test: {accTest * 100:.3f}%')

## COMPARACION DE CLASIFICADORES

Ahora, para la comparación de los clasificadores, vamos a hacer uso de la validación, de tal manera que un 20% de los datos 
del test serán la validación la cual va a determinar el mejor clasificador.

In [ ]:
clasificadores = [Regresion_Logistica, Naive_Bayes, Redes_Neuronales, Decision_Tree, gridOVO, gridOVA, Bagging, Ada_Boost_best, Random_Forest_best]
nombres_clasificadores = ['Regresion Logistica', 'Naive Bayes', 'Redes Neuronales', 'Decision Tree','OVO','OVA', 'Bagging', 'Ada Boost', 'Random Forest']

best_val = 0
accTrain = np.zeros(len(clasificadores))
accTest = np.zeros(len(clasificadores))
accVal = np.zeros(len(clasificadores))
precisiones = []

X_test_comp, X_val_comp, y_test_comp, y_val_comp = train_test_split(pca_vector_test, y_test, test_size=0.2, random_state=0)


for i, clasificador in enumerate(clasificadores):
    accTrain[i] = metrics.accuracy_score(y_train, clasificador.predict(pca_vector_train))
    accTest[i] = metrics.accuracy_score(y_test_comp, clasificador.predict(X_test_comp))
    accVal[i] = metrics.accuracy_score(y_val_comp, clasificador.predict(X_val_comp))
    precisiones.append((accTrain[i],accVal[i],accTest[i]))

    if accVal[i] > best_val:
        best_val = accVal[i]
        best_test = accTest[i]
        best_clasificador = clasificador
        nombre_best_clasificador = nombres_clasificadores[i]

# Obtener los datos en un formato adecuado para mostrarlos en un diagrama de barras
datosPD = np.hstack((np.array(accTrain).reshape(-1,1), np.array(accVal).reshape(-1,1), np.array(accTest).reshape(-1,1)))
datosPanda = pd.DataFrame(data=precisiones, index=nombres_clasificadores, columns=['Train', 'Validation', 'Test'])
print(datosPanda)

# Mostrar los resultados en un diagrama de barras
plt.figure(figsize=(20, 6))

plt.subplot(1, 3, 1)
plt.title('Precisión en entrenamiento')
bars = plt.barh(nombres_clasificadores, datosPanda['Train'])
plt.xlim([0, 0.6])
for bar in bars:
    width = bar.get_width()
    plt.text(0.01, bar.get_y() + bar.get_height()/2, f'{width:.2%}', va='center', ha='left')

plt.subplot(1, 3, 2)
plt.title('Precisión en validación')
bars = plt.barh(nombres_clasificadores, datosPanda['Validation'])
plt.xlim([0, 0.6])
for bar in bars:
    width = bar.get_width()
    plt.text(0.01, bar.get_y() + bar.get_height()/2, f'{width:.2%}', va='center', ha='left')

plt.subplot(1, 3, 3)
plt.title('Precisión en test')
bars = plt.barh(nombres_clasificadores, datosPanda['Test'])
plt.xlim([0, 0.6])
for bar in bars:
    width = bar.get_width()
    plt.text(0.01, bar.get_y() + bar.get_height()/2, f'{width:.2%}', va='center', ha='left')

plt.tight_layout()
plt.show()
print(f'Mejor modelo: {nombre_best_clasificador}')
print(f'Precisión en test: {round(best_test*100,3)}%')

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = best_clasificador.predict(pca_vector_test)

# Crear la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Configurar el estilo de la matriz de confusión
plt.figure(figsize=(10, 8))

sns.set(font_scale=1.2)
sns.heatmap(cm, annot=True, fmt='g', cmap='coolwarm', cbar=False, 
            xticklabels=['Gato', 'Elefante', 'Caballo', 'Araña'], 
            yticklabels=['Gato', 'Elefante', 'Caballo', 'Araña'])

plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores reales')
plt.show()